<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/ulmfit/ULMFitTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Montando o Encoder

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *
import pandas as pd
from pathlib import Path
from datetime import datetime
from google.colab import drive

In [ ]:
train_on_gpu=torch.cuda.is_available()
if(train_on_gpu):
    print('Treinamento em GPU :-)')
else:
    print('Sem disponibilidade de GPU, usando CPU. :-(')


Treinamento em GPU :-)


# Preparando os dados.

In [ ]:
#nos Correios
#path = '/opt/app/jupyter/GDMT'
#df = pd.read_csv('twitterUMLFit.csv')
#df = df.sample(frac=0.5)
#df.count()


In [ ]:
#collab
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks
data_path ='twitterUMLFit.csv'
df = pd.read_csv(data_path)
print(df.count())
path = '/gdrive/My Drive/Colab Notebooks/'


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


id_post                584513
data_post              584513
is_retweeted           584513
latitude               584513
longitude              584513
referencia_pesquisa    584513
retweet_count          584513
login_usuario          584513
mensagem               584513
link                    63964
kmeans                      0
dtype: int64


In [ ]:
%cd /gdrive/My\ Drive/Colab\ Notebooks/models
!ls *.csv -la

/gdrive/My Drive/Colab Notebooks/models
ls: cannot access '*.csv': No such file or directory


In [ ]:
#df.sample(frac = 0.3, random_state = 1234)
#print(df.count())

# Carregando os dados - montando o databunch

In [ ]:

bs = 128
lang = 'pt'
data = (TextList.from_csv(path,'twitterUMLFit.csv', cols='mensagem')
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
data.save(f'{lang}_TWTdatabunch')
len(data.vocab.itos),len(data.train_ds)

(60000, 526062)

In [ ]:
bs = 128
lang = 'pt'
data = load_data(path, f'{lang}_TWTdatabunch', bs=bs)

In [ ]:
data.show_batch()

idx,text
0,"chega ao xxmaj brasil é independente … https : / / t.co / xxunk xxbos xxup r$ 9,90 \n xxmaj enviamos via xxmaj correios para todo o xxmaj brasil . \n - xxmaj caso esteja situado em xxmaj são xxmaj gonçalo ou regiões próximas , poderá … https : / / t.co / xxunk xxbos i 'm at xxmaj correios in xxmaj indaiatuba , xxmaj são xxmaj paulo"
1,que depende dos xxmaj correios xxbos xxup meu xxup óculos xxup quebrou e os xxup correios xxup perdeu xxup minha xxup encomenda- xxup vlog \n https : / / t.co / vypoiqxivr xxbos xxunk xxmaj depende dos correios anjo . xxmaj normalmente de uma a duas semanas xxbos xxunk xxmaj vamo . xxmaj de correios xxbos amanhã eu tinha que sair pra resolver muita coisa mas vou ter que
2,"/ t.co / xxunk xxbos xxmaj eu vou processar os correios se eu não achar minha encomenda , esses irresponsáveis nunca vi esse negócio de fazer … https : / / t.co / xxunk xxbos xxunk xxrep 4 a h xxup que xxup meu xxup convite xxup não xxup chegou xxup na xxup minha xxup caixinha xxup de xxup correios xxup ainda . xxup cadê xxbos xxunk xxunk xxmaj muito"
3,"várias cores pra o frete valer a pena , comprei 60 gramas de mica em pó … https : / / t.co / xxunk xxbos xxmaj com reestruturação dos xxmaj correios , cortes podem chegar a 20 mil . xxmaj presidente da estatal disse à xxmaj folha que número ideal … https : / / t.co / xxunk xxbos xxmaj correios passa por reestruturação e cortes podem chegar a 20"
4,"jadlog na lista de transportadora com entrega lixo igual a xxmaj total xxmaj express ... toda reclamação que já f … https : / / t.co / xxunk xxbos @roxmo @nirdop é uma parceria de vagabundos , alfândega e correios . xxmaj quando não tão de greve , tão de atestado médico . … https : / / t.co / xxunk xxbos @edupezz @rodrigo4334 @thyagojuliana @roxmo xxmaj portanto , no"


# Usando um modelo já treinado.

In [ ]:
#o modelo foi treinado para o inglês. Como os dados estão em português, utilizaremos a flag pretrained = False
#e rodaremos um ciclo de treinamento do modelo para o português.
if not train_on_gpu:
  learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False)#.to_fp16()
else:
  learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [ ]:
lr = 1e-2 #manteremos o 0,01. Testei com 0,001 e o resultado não foi satisfatório (o caimento foi mais lento e a acurácia teve piora).

In [ ]:
inicio = datetime.now()
learn.unfreeze() #como iremos treinar o modelo para definirmos novos parâmetros, liberarmos os parâmetros de treinamento para redefinição (descongelamento dos parâmetros de treinamento)
learn.fit_one_cycle(11, lr, moms=(0.8,0.7)) 
print('duracao: ', datetime.now()-inicio)
'''rodando para 11 iterações e lr = 1e-2

0	3.511818	3.387745	0.430466	08:40
1	3.192427	3.080928	0.456455	08:44
2	3.141455	3.042238	0.458198	08:44
3	3.077677	2.977442	0.464996	08:44
4	3.010370	2.914681	0.472557	08:44
5	2.916011	2.850265	0.479905	08:44
6	2.833328	2.787559	0.487808	08:44
7	2.742976	2.729383	0.495477	08:44
8	2.646871	2.684921	0.501621	08:44
9	2.565699	2.662056	0.505357	08:44
10	2.527627	2.659061	0.505916	08:44

Tempo: duracao:  1:36:05.089229 '''


epoch,train_loss,valid_loss,accuracy,time
0,3.526235,3.395679,0.429953,07:37
1,3.183123,3.083472,0.456208,07:40
2,3.155166,3.046926,0.457276,07:40
3,3.086245,2.987351,0.463695,07:40
4,3.012007,2.918219,0.471969,07:41


In [ ]:
#Salvando o modelo pre-treinado e o vocabulário (vocab) com as mensagens do twitter.

In [ ]:
lm_fns = [f'{lang}_TWT_wt', f'{lang}_TWT_wt_vocab']
pathlms = Path(path+'/models')
learn.to_fp32().save(pathlms/lm_fns[0], with_opt=False)
learn.data.vocab.save(pathlms/(lm_fns[1] + '.pkl'))


In [ ]:
#ENCODER

In [ ]:
#%cd /gdrive/My\ Drive/Colab\ Notebooks
train_df = pd.read_csv('twitterclusterizado2.csv')
test_df = pd.read_csv('tweetssetembro.csv')
df = pd.concat([train_df,test_df], sort=False)

In [ ]:
bs= 128
data_lm = (TextList.from_df(df, path, cols='mensagem')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))


In [ ]:
data_lm.save(f'{lang}_twt_lm')

In [ ]:
lang='pt'
lm_fns = [f'{lang}_TWT_wt', f'{lang}_TWT_wt_vocab']

learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [ ]:
lr = 1e-3
lr *= bs/48

In [ ]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [ ]:
pathlms = Path(path+'/models')
learn_lm.save(pathlms/f'{lang}_TWT_fine_tuned')
learn_lm.save_encoder(pathlms/f'{lang}_TWT_fine_tuned_enc')